### Connect to DuckDB + load table

In [1]:
from pathlib import Path
import duckdb
import pandas as pd

project_root = Path.cwd().resolve()
while not (project_root / "Day-1").exists():
    if project_root == project_root.parent:
        raise FileNotFoundError("Could not find project root containing Day-1.")
    project_root = project_root.parent

db_path = project_root / "Day-1" / "data" / "warehouse" / "day1.duckdb"
con = duckdb.connect(str(db_path))
print("Connected to:", db_path)

df = con.execute("SELECT * FROM gold_diabetes_features_v1").df()
print("df shape:", df.shape)
df.head()


Connected to: C:\Users\sarfo\Dropbox\Courses\Data Science\30-days-of-data-science\Day-1\data\warehouse\day1.duckdb
df shape: (101766, 20)


,encounter_id,person_id,label,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,diabetesMed,change,insulin,A1Cresult
0,2278392,8222157,0,1,41,0,1,0,0,0,Caucasian,Female,[0-10),6,25,1,No,No,No,None
1,149190,55629189,0,3,59,0,18,0,0,0,Caucasian,Female,[10-20),1,1,7,Yes,Ch,Up,None
2,64410,86047875,0,2,11,5,13,2,0,1,AfricanAmerican,Female,[20-30),1,1,7,Yes,No,No,None
3,500364,82442376,0,2,44,1,16,0,0,0,Caucasian,Male,[30-40),1,1,7,Yes,Ch,Up,None
4,16680,42519267,0,1,51,0,8,0,0,0,Caucasian,Male,[40-50),1,1,7,Yes,Ch,Steady,None


### Train/valid/test split (patient-safe, same 60/20/20 logic)

In [2]:
from sklearn.model_selection import GroupShuffleSplit

y = df["label"].astype(int)
groups = df["person_id"]

# Hold out TEST (20%)
gss1 = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
idx_trainval, idx_test = next(gss1.split(df, y, groups=groups))

df_trainval = df.iloc[idx_trainval].copy()
df_test = df.iloc[idx_test].copy()

# Split TRAIN vs VALID inside trainval (valid is 25% of trainval -> ~20% of total)
y_tv = df_trainval["label"].astype(int)
g_tv = df_trainval["person_id"]

gss2 = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
idx_train, idx_valid = next(gss2.split(df_trainval, y_tv, groups=g_tv))

df_train = df_trainval.iloc[idx_train].copy()
df_valid = df_trainval.iloc[idx_valid].copy()

print("Train rows:", df_train.shape[0], "Valid rows:", df_valid.shape[0], "Test rows:", df_test.shape[0])
print("Prevalence train/valid/test:",
      float(df_train["label"].mean()),
      float(df_valid["label"].mean()),
      float(df_test["label"].mean()))

# overlap checks (must be 0)
print("Overlap train-valid:", len(set(df_train["person_id"]) & set(df_valid["person_id"])))
print("Overlap train-test:", len(set(df_train["person_id"]) & set(df_test["person_id"])))
print("Overlap valid-test:", len(set(df_valid["person_id"]) & set(df_test["person_id"])))


Train rows: 60988 Valid rows: 20625 Test rows: 20153
Prevalence train/valid/test: 0.11218600380402702 0.11461818181818181 0.10673348881059892
Overlap train-valid: 0
Overlap train-test: 0
Overlap valid-test: 0


### Preprocessing (dense one-hot for hist_gb)

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

id_cols = ["encounter_id", "person_id", "label"]
feature_cols = [c for c in df.columns if c not in id_cols]

numeric_cols = [
    "time_in_hospital", "num_lab_procedures", "num_procedures", "num_medications",
    "number_outpatient", "number_emergency", "number_inpatient"
]
categorical_cols = [c for c in feature_cols if c not in numeric_cols]

X_train = df_train[feature_cols]
y_train = df_train["label"].astype(int)

X_valid = df_valid[feature_cols]
y_valid = df_valid["label"].astype(int)

X_test = df_test[feature_cols]
y_test = df_test["label"].astype(int)

def make_dense_ohe():
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)

prep_tree_dense = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("impute", SimpleImputer(strategy="median"))]), numeric_cols),
        ("cat", Pipeline([("impute", SimpleImputer(strategy="most_frequent")),
                          ("onehot", make_dense_ohe())]), categorical_cols),
    ]
)


### Fit the Day-4 winner (hist_gb) and evaluate baseline

In [5]:
import numpy as np
from sklearn.metrics import average_precision_score, roc_auc_score, brier_score_loss, log_loss
import numpy as np

def metrics(y_true, p):
    y_true = np.asarray(y_true)
    p = np.asarray(p)

    # prevent log(0) by clipping probabilities
    p_clip = np.clip(p, 1e-15, 1 - 1e-15)

    return {
        "prevalence": float(y_true.mean()),
        "mean_p": float(p.mean()),
        "median_p": float(np.median(p)),
        "pr_auc": float(average_precision_score(y_true, p)),
        "roc_auc": float(roc_auc_score(y_true, p)),
        "brier": float(brier_score_loss(y_true, p)),
        "logloss": float(log_loss(y_true, p_clip, labels=[0, 1])),
    }


m_valid_base = metrics(y_valid, p_valid)
m_test_base  = metrics(y_test, p_test)

print("BASE (uncalibrated) valid:", m_valid_base)
print("BASE (uncalibrated) test :", m_test_base)


BASE (uncalibrated) valid: {'prevalence': 0.11461818181818181, 'mean_p': 0.11277848922639576, 'median_p': 0.09866236945430058, 'pr_auc': 0.22467831773854963, 'roc_auc': 0.6677610282599987, 'brier': 0.09683001463055108, 'logloss': 0.33576765492464755}
BASE (uncalibrated) test : {'prevalence': 0.10673348881059892, 'mean_p': 0.1111784920382252, 'median_p': 0.0975009866650095, 'pr_auc': 0.2078325210559714, 'roc_auc': 0.6666140354981995, 'brier': 0.0913437944101995, 'logloss': 0.32108924251898663}


### Calibration (Sigmoid + Isotonic) using VALID

Calibration is “probability fixing”. It usually keeps ranking similar, but improves probability quality (Brier/logloss).

In [7]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression

# --- helper: safe clip ---
def clip01(p, eps=1e-15):
    p = np.asarray(p)
    return np.clip(p, eps, 1 - eps)

# --- 1) SIGMOID calibration (Platt scaling) on VALID only ---
# We calibrate by fitting a logistic regression on the logit of p_valid.
p_valid_c = clip01(p_valid)
p_test_c  = clip01(p_test)

z_valid = np.log(p_valid_c / (1 - p_valid_c)).reshape(-1, 1)
z_test  = np.log(p_test_c  / (1 - p_test_c)).reshape(-1, 1)

platt = LogisticRegression(solver="lbfgs", C=1e6, max_iter=2000)  # very weak regularization
platt.fit(z_valid, y_valid)

p_valid_sig = platt.predict_proba(z_valid)[:, 1]
p_test_sig  = platt.predict_proba(z_test)[:, 1]

# --- 2) ISOTONIC calibration on VALID only ---
iso = IsotonicRegression(out_of_bounds="clip")
iso.fit(p_valid, y_valid)           # fit mapping p -> y on VALID

p_valid_iso = iso.transform(p_valid)
p_test_iso  = iso.transform(p_test)

# --- evaluate ---
m_valid_sig = metrics(y_valid, p_valid_sig)
m_test_sig  = metrics(y_test,  p_test_sig)

m_valid_iso = metrics(y_valid, p_valid_iso)
m_test_iso  = metrics(y_test,  p_test_iso)

print("SIGMOID (Platt) valid:", m_valid_sig)
print("SIGMOID (Platt) test :", m_test_sig)
print("ISOTONIC valid:", m_valid_iso)
print("ISOTONIC test :", m_test_iso)


SIGMOID (Platt) valid: {'prevalence': 0.11461818181818181, 'mean_p': 0.11462417649952247, 'median_p': 0.09924107224593008, 'pr_auc': 0.22467831773854963, 'roc_auc': 0.6677610282599987, 'brier': 0.09681639063245714, 'logloss': 0.3357148386561877}
SIGMOID (Platt) test : {'prevalence': 0.10673348881059892, 'mean_p': 0.1129350547535146, 'median_p': 0.09802124987096311, 'pr_auc': 0.2078325210559714, 'roc_auc': 0.6666140354981995, 'brier': 0.09137285830812976, 'logloss': 0.3211598654462384}
ISOTONIC valid: {'prevalence': 0.11461818181818181, 'mean_p': 0.11461818181818181, 'median_p': 0.10289236605026078, 'pr_auc': 0.21822358537800177, 'roc_auc': 0.6706074108172614, 'brier': 0.09648336720036713, 'logloss': 0.33432929598511896}
ISOTONIC test : {'prevalence': 0.10673348881059892, 'mean_p': 0.1128547497241375, 'median_p': 0.10131332082551595, 'pr_auc': 0.19904850431304646, 'roc_auc': 0.6666061459879116, 'brier': 0.09135924871272744, 'logloss': 0.3259030906318809}


### Expected Calibration Error (ECE) (simple and useful)

In [8]:
import numpy as np

def ece(y_true, p, n_bins=10):
    y_true = np.asarray(y_true)
    p = np.asarray(p)
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    idx = np.digitize(p, bins) - 1
    ece_val = 0.0
    n = len(p)
    for b in range(n_bins):
        mask = (idx == b)
        if mask.sum() == 0:
            continue
        conf = p[mask].mean()
        acc  = y_true[mask].mean()
        ece_val += (mask.sum() / n) * abs(acc - conf)
    return float(ece_val)

print("ECE base:", ece(y_valid, p_valid))
print("ECE sigmoid:", ece(y_valid, p_valid_sig))
print("ECE isotonic:", ece(y_valid, p_valid_iso))


ECE base: 0.004044470923893383
ECE sigmoid: 0.0037983936764188473
ECE isotonic: 2.1262453077669665e-18


### Choose the calibration method (pick best by VALID Brier)

In [13]:
candidates = [
    ("base", p_valid, p_test, m_valid_base, m_test_base),
    ("sigmoid", p_valid_sig, p_test_sig, m_valid_sig, m_test_sig),
    ("isotonic", p_valid_iso, p_test_iso, m_valid_iso, m_test_iso),
]

best = min(candidates, key=lambda t: t[3]["brier"])  # smallest valid Brier
best_name, p_valid_best, p_test_best, m_valid_best, m_test_best = best

print("Best calibration by VALID Brier:", best_name)
print("VALID metrics:", m_valid_best)
print("TEST metrics :", m_test_best)


Best calibration by VALID Brier: isotonic
VALID metrics: {'prevalence': 0.11461818181818181, 'mean_p': 0.11461818181818181, 'median_p': 0.10289236605026078, 'pr_auc': 0.21822358537800177, 'roc_auc': 0.6706074108172614, 'brier': 0.09648336720036713, 'logloss': 0.33432929598511896}
TEST metrics : {'prevalence': 0.10673348881059892, 'mean_p': 0.1128547497241375, 'median_p': 0.10131332082551595, 'pr_auc': 0.19904850431304646, 'roc_auc': 0.6666061459879116, 'brier': 0.09135924871272744, 'logloss': 0.3259030906318809}


### Turn probabilities into an action rule (capacity targeting)

If you can only intervene on, say, the top 10% highest risk, this is the clean operational rule.

In [14]:
import pandas as pd
import numpy as np

def topk_summary(y_true, p, frac=0.10):
    y_true = np.asarray(y_true)
    n = len(y_true)
    k = max(1, int(np.floor(frac*n)))
    order = np.argsort(-p)
    idx = order[:k]
    return {
        "frac": frac,
        "k": k,
        "captured": int(y_true[idx].sum()),
        "precision_at_k": float(y_true[idx].mean()),
        "threshold": float(np.quantile(p, 1-frac))
    }

for frac in [0.01, 0.05, 0.10, 0.20]:
    print("TEST", topk_summary(y_test, p_test_best, frac=frac))


TEST {'frac': 0.01, 'k': 201, 'captured': 81, 'precision_at_k': 0.40298507462686567, 'threshold': 0.37264150943396224}
TEST {'frac': 0.05, 'k': 1007, 'captured': 309, 'precision_at_k': 0.30685203574975173, 'threshold': 0.23628691983122363}
TEST {'frac': 0.1, 'k': 2015, 'captured': 499, 'precision_at_k': 0.24764267990074443, 'threshold': 0.19258295380611581}
TEST {'frac': 0.2, 'k': 4030, 'captured': 828, 'precision_at_k': 0.2054590570719603, 'threshold': 0.14360770577933452}


### Save Day-5 artifacts

In [11]:
import json
from pathlib import Path

reports_dir = project_root / "Day-5" / "reports"
reports_dir.mkdir(parents=True, exist_ok=True)

out = {
    "split": {"method": "GroupShuffleSplit by person_id", "approx": "60/20/20", "random_state": 42},
    "baseline_uncalibrated": {"valid": m_valid_base, "test": m_test_base},
    "sigmoid": {"valid": m_valid_sig, "test": m_test_sig},
    "isotonic": {"valid": m_valid_iso, "test": m_test_iso},
    "best_by_valid_brier": {"name": best_name, "valid": m_valid_best, "test": m_test_best},
    "ece_valid": {
        "base": ece(y_valid, p_valid),
        "sigmoid": ece(y_valid, p_valid_sig),
        "isotonic": ece(y_valid, p_valid_iso),
    }
}

with open(reports_dir / "DAY05_calibration_metrics.json", "w", encoding="utf-8") as f:
    json.dump(out, f, indent=2)

# Save top-200 predictions on TEST using best probabilities
top = df_test[["encounter_id", "person_id", "label"]].copy()
top["p_hat"] = p_test_best
top = top.sort_values("p_hat", ascending=False).head(200)
top.to_csv(reports_dir / "DAY05_top200_test_predictions_best.csv", index=False)

print("Saved:", reports_dir / "DAY05_calibration_metrics.json")
print("Saved:", reports_dir / "DAY05_top200_test_predictions_best.csv")


Saved: C:\Users\sarfo\Dropbox\Courses\Data Science\30-days-of-data-science\Day-5\reports\DAY05_calibration_metrics.json
Saved: C:\Users\sarfo\Dropbox\Courses\Data Science\30-days-of-data-science\Day-5\reports\DAY05_top200_test_predictions_best.csv


Close DuckDB:

In [12]:
con.close()
print("DuckDB closed.")

DuckDB closed.
